In [ ]:
import os
import sys
import logging

import numpy as np
import pandas as pd
from sagemaker.local import LocalSession
import sagemaker
from sagemaker.pytorch import PyTorch
import torch

In [ ]:
session = LocalSession()
session.config = {"local": {"local_code": True}}
bucket = sagemaker_session.default_bucket()
role = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001"
region = sagemaker_session.region
train_input_path = f"s3://{bucket}/imdb/data/train.csv"
test_input_path = f"s3://{bucket}/imdb/data/test.csv"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

## Training

In [ ]:
hyperparameters = {
    "epochs": 1,
    "train_batch_size": 32,
    "model_name": "distilbert-base-uncased",
}
estimator = PyTorch(
    entry_point="train.py",
    source_dir="source",
    role=role,
    framework_version="1.7.1",
    py_version="py3",
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
)
estimator.fit({'train': train_input_path, 'test': test_input_path})

## Inference

In [ ]:
# model_data = estimator.model_data
model_data = 's3://sagemaker-eu-central-1-611215368770/pytorch-training-2021-06-01-16-57-40-514/model.tar.gz'
print(model_data)

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data="s3://sagemaker-eu-central-1-611215368770/pytorch-training-2021-06-01-09-20-56-523/model.tar.gz",
    role=role,
    framework_version="1.7.1",
    source_dir="source",
    py_version="py3",
    entry_point="inference.py",
)


### Endpoint

In [ ]:
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type="local",
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.StringDeserializer(),
)

In [ ]:
input_1 = "this is a very good movie"
predictor.predict(input_1)

In [ ]:
input_2 = ['this movie sucks', 'this movie is ok']
predictor.predict(input_2)

In [ ]:
input_3 = ['such a terrible movie', 'what a great movie', 'omg best movie ever']
predictor.predict(input_3)

In [ ]:
predictor.delete_endpoint()

### Batch Transform

In [ ]:
transformer = pytorch_model.transformer(instance_count=1, instance_type="local", accept='text/csv')

In [ ]:
transformer.transform(test_input_path, content_type='text/csv', split_type='Line')

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./

In [ ]:
res = pd.read_csv(f"{transformer.output_path.split('/')[-1]}/test.csv.out", header=None)
res

### Local

In [ ]:
%%sh -s $estimator.model_data
rm -rf model
mkdir model
aws s3 cp $1 model/ 
tar xvzf model/model.tar.gz --directory ./model

In [ ]:
# pytorch_model.bin stores fine-tuned huggingface
# tokenizer_config.json stores tokenizer used for training

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

model = AutoModelForSequenceClassification.from_pretrained('model')
tokenizer = AutoTokenizer.from_pretrained('model')

In [ ]:
inputs = ['this is a terrific movie']

In [ ]:
tokenized_input = tokenizer(inputs, truncation=True, padding=True)

In [ ]:
input_ids = torch.Tensor(tokenized_input['input_ids']).long()
attention_mask = torch.Tensor(tokenized_input['attention_mask']).long()

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(input_ids, attention_mask).logits
    res = np.argmax(logits)
    print('logits:', logits, 'neg/pos:', res)